In [8]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import os

In [9]:
base_path = 'E:/deepfake/deepfake_resnet50/Dataset/'

In [10]:
base_path = 'E:/deepfake/deepfake_resnet50/Dataset/'
input_shape = (128, 128, 3)
batch_size = 64
augmentation_factor = 2  # Augment the data by a factor of 2
small_train_steps = 1000 // (batch_size * augmentation_factor)

In [11]:
def build_model():
    densenet = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    model = Sequential([
        densenet,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(2, activation='softmax')
    ])
    model.compile(
        optimizer=Adam(lr=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 24,639,874
Trainable params: 24,585,730
No

C:\Users\Admin\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [12]:
# Define an augmented data generator
augmented_generator = ImageDataGenerator(
    rescale=1. / 255.,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [13]:
# Create augmented training data generator
augmented_train_flow = augmented_generator.flow_from_directory(
    os.path.join(base_path, 'Train'),
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 140002 images belonging to 2 classes.


In [14]:
# Validation data generator (no augmentation)
valid_generator = ImageDataGenerator(rescale=1. / 255.)
valid_flow = valid_generator.flow_from_directory(
    os.path.join(base_path, 'Validation'),
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 39428 images belonging to 2 classes.


In [15]:
# Define a custom callback to compute and print confusion matrix
class PredictionCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(valid_flow[0][0])
        y_test = valid_flow[0][1]
        y_pred_labels = np.argmax(y_pred, axis=1)
        y_test_labels = np.argmax(y_test, axis=1)
        cfm = confusion_matrix(y_test_labels, y_pred_labels)
        print(cfm)
        print(y_pred[0], y_test[0])

In [17]:
# Calculate the number of validation steps
valid_steps = 10000 // batch_size

# Train the model with augmented data
history = model.fit(
    augmented_train_flow,
    epochs=10,
    steps_per_epoch=small_train_steps,
    validation_data=valid_flow,
    validation_steps=valid_steps,
    callbacks=[PredictionCallback()]
)

Epoch 1/10
2/2 [==============================] - 4s 1s/steploss: 1.4914 - accuracy: 0.
[[ 0 27]
 [ 0 37]]
[5.8830176e-19 1.0000000e+00] [1. 0.]
7/7 [==============================] - 346s 56s/step - loss: 1.4914 - accuracy: 0.6071 - val_loss: 21.4357 - val_accuracy: 0.4925
Epoch 2/10
2/2 [==============================] - 2s 1s/steploss: 1.0601 - accuracy: 0.
[[29  0]
 [35  0]]
[1. 0.] [1. 0.]
7/7 [==============================] - 407s 66s/step - loss: 1.0601 - accuracy: 0.6763 - val_loss: 105.3194 - val_accuracy: 0.5030
Epoch 3/10
2/2 [==============================] - 2s 1s/steploss: 0.6277 - accuracy: 0.7
[[ 0 34]
 [ 0 30]]
[0. 1.] [0. 1.]
7/7 [==============================] - 424s 69s/step - loss: 0.6277 - accuracy: 0.7054 - val_loss: 2006.0413 - val_accuracy: 0.4999
Epoch 4/10
2/2 [==============================] - 2s 1s/steploss: 0.6013 - accuracy: 0.
[[39  0]
 [25  0]]
[1. 0.] [0. 1.]
7/7 [==============================] - 401s 65s/step - loss: 0.6013 - accuracy: 0.7299 - val

In [19]:
model.save('my_model.h5')